In [1]:
import os
# os.chdir('../')
import pandas as pd
import numpy as np
from DeepMTP.dataset import load_process_MLC, load_process_MTR, load_process_DP, process_dummy_MLC, process_dummy_MTR, process_dummy_DP, load_process_MC, load_process_MTL
from DeepMTP.utils.data_utils import data_process, BaseDataset
from DeepMTP.utils.tests import check_mlc_results, check_mtr_results, check_dp_results, check_mtl_results
from DeepMTP.main import DeepMTP
from DeepMTP.utils.utils import generate_config
from tqdm import tqdm
from PIL import Image

/home/dimitriosi/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Dyadic Prediction
<img src="../images/dp_example.png"/>

# load-process-split

In [2]:
# load dataset
data = load_process_DP('ern')
# process and split
train, val, test, data_info = data_process(data, validation_setting='D', verbose=True)
# sanity check
check_dp_results(train, val, test)

Processing...
Done
Interaction file: 2d numpy array format detected
Interaction file: checking format consistency... Passed
Interaction file: checking instance id format consistency... Passed
Interaction file: checking target id type consistency... Passed

Interaction file: checking target variable type consistency... Passed
Automatically detected type of target variable type: binary

-- Test set was not provided, could not detect if novel instances exist or not 
-- Test set was not provided, could not detect if novel targets exist or not 

Instance features file: processing features... Done
Instance features file: processing features... Done

Cross input consistency for (numpy) interaction data and instance features checks out
Cross input consistency for (numpy) interaction data and target features checks out

Splitting train to train-test according to validation setting D... Done
Splitting train to train-val according to validation setting D... Done
Checking if DP splitting results a

In [6]:
data_info

{'detected_validation_setting': 'D',
 'detected_problem_mode': 'classification',
 'instance_branch_input_dim': 445,
 'target_branch_input_dim': 445}

# Configure and train network

In [7]:
config = generate_config(    
    instance_branch_input_dim = data_info['instance_branch_input_dim'],
    target_branch_input_dim = data_info['target_branch_input_dim'],
    validation_setting = data_info['detected_validation_setting'],
    general_architecture_version = 'dot_product',
    problem_mode = data_info['detected_problem_mode'],
    learning_rate = 0.001,
    decay = 0,
    batch_norm = False,
    dropout_rate = 0,
    momentum = 0.9,
    weighted_loss = False,
    compute_mode = 'cuda:1',
    train_batchsize = 512,
    val_batchsize = 512,
    num_epochs = 50,
    num_workers = 8,
    metrics = ['aupr', 'auroc'],
    metrics_average = ['macro'],
    patience = 10,

    evaluate_train = True,
    evaluate_val = True,

    verbose = True,
    results_verbose = False,
    use_early_stopping = True,
    use_tensorboard_logger = True,
    wandb_project_name = 'DeepMTP_v2',
    wandb_project_entity = 'diliadis',
    metric_to_optimize_early_stopping = 'loss',
    metric_to_optimize_best_epoch_selection = 'loss',

    instance_branch_architecture = 'MLP',
    use_instance_features = False,
    instance_branch_params = {
        'instance_branch_nodes_reducing_factor': 2,
        'instance_branch_nodes_per_layer': [512, 256],
        'instance_branch_layers': None,
        # 'instance_branch_conv_architecture': 'resnet',
        # 'instance_branch_conv_architecture_version': 'resnet101',
        # 'instance_branch_conv_architecture_dense_layers': 1,
        # 'instance_branch_conv_architecture_last_layer_trained': 'last',
    },

    target_branch_architecture = 'MLP',
    use_target_features = False,
    target_branch_params = {
        'target_branch_nodes_reducing_factor': 2,
        'target_branch_nodes_per_layer': [512, 256],
        'target_branch_layers': None,
        # 'target_branch_conv_architecture': 'resnet',
        # 'target_branch_conv_architecture_version': 'resnet101',
        # 'target_branch_conv_architecture_dense_layers': 1,
        # 'target_branch_conv_architecture_last_layer_trained': 'last',
    },

    embedding_size = 30,
    comb_mlp_nodes_reducing_factor = 2,
    comb_mlp_nodes_per_layer = [2048, 2048, 2048],
    comb_mlp_layers = None, 

    save_model = True,

    eval_every_n_epochs = 10,

    additional_info = {})

Micro-averaging is the adviced averaging option for validation setting D. The micro option will be included in the results


In [8]:
# initialize model
model = DeepMTP(config)
print(str(model.deepMTP_model))
# train, validate, test
validation_results = model.train(train, val, test)

Selected device: cuda:1


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


TwoBranchDotProductModel(
  (instance_branch_model): MLP(
    (predictor): ModuleList(
      (0): Linear(in_features=445, out_features=512, bias=True)
      (1): LeakyReLU(negative_slope=0.01)
      (2): Linear(in_features=512, out_features=256, bias=True)
      (3): LeakyReLU(negative_slope=0.01)
      (4): Linear(in_features=256, out_features=30, bias=True)
      (5): LeakyReLU(negative_slope=0.01)
    )
  )
  (target_branch_model): MLP(
    (predictor): ModuleList(
      (0): Linear(in_features=445, out_features=512, bias=True)
      (1): LeakyReLU(negative_slope=0.01)
      (2): Linear(in_features=512, out_features=256, bias=True)
      (3): LeakyReLU(negative_slope=0.01)
      (4): Linear(in_features=256, out_features=30, bias=True)
      (5): LeakyReLU(negative_slope=0.01)
    )
  )
)
Starting training...


wandb: Currently logged in as: diliadis. Use `wandb login --relogin` to force relogin


Epoch:0... Warning: instance120 has 1 unique true values
Done
  Validating... Calculating val performance... Done
Done
Epoch:1... Done
  Validating... Done
-----------------------------EarlyStopping counter: 1 out of 10---------------------- best epoch currently 0
Epoch:2... Done
  Validating... Done
-----------------------------EarlyStopping counter: 2 out of 10---------------------- best epoch currently 0
Epoch:3... Done
  Validating... Done
-----------------------------EarlyStopping counter: 3 out of 10---------------------- best epoch currently 0
Epoch:4... Done
  Validating... Done
-----------------------------EarlyStopping counter: 4 out of 10---------------------- best epoch currently 0
Epoch:5... Done
  Validating... Done
-----------------------------EarlyStopping counter: 5 out of 10---------------------- best epoch currently 0
Epoch:6... Done
  Validating... Done
-----------------------------EarlyStopping counter: 6 out of 10---------------------- best epoch currently 0
Epoch

test_aupr_macro,▁
test_aupr_micro,▁
test_auroc_macro,▁
test_auroc_micro,▁
train_aupr_macro,█▁
train_aupr_micro,▁█
train_auroc_macro,█▁
train_auroc_micro,▁█
train_loss,▁██████████
val_aupr_macro,▁▁
val_aupr_micro,▁▁


+-------+--------+---------+------------+------------+-------------+-------------+
|  mode | #epoch |   loss  | aupr_macro | aupr_micro | auroc_macro | auroc_micro |
+-------+--------+---------+------------+------------+-------------+-------------+
| train |   0    | 75.9355 |   0.0225   |   0.0248   |    0.4942   |    0.5307   |
| train |   1    | 97.9912 |     -      |     -      |      -      |      -      |
| train |   2    | 97.9916 |     -      |     -      |      -      |      -      |
| train |   3    | 97.9916 |     -      |     -      |      -      |      -      |
| train |   4    | 97.9923 |     -      |     -      |      -      |      -      |
| train |   5    | 97.9921 |     -      |     -      |      -      |      -      |
| train |   6    | 97.9923 |     -      |     -      |      -      |      -      |
| train |   7    | 97.9912 |     -      |     -      |      -      |      -      |
| train |   8    | 97.9925 |     -      |     -      |      -      |      -      |
| tr

# Inference

In [ ]:
# generate predictions from the trained model
results, preds = model.predict(train, return_predictions=True ,verbose=True)

# Continue training model

In [ ]:
# this is a minimal configuration needed for HPO methods like Hyperband
config = {
    'verbose': True,
    'num_epochs': 20,
    'num_workers': 8,
    # 'wandb_project_name': None,
    # 'wandb_project_entity': None,
    # 'use_tensorboard_logger': False
}
# initialize the model and load the pretrained weights etc.
new_model = DeepMTP(config, 'add path to model.pt')

new_model.train(train, val, test)